In [1]:
import pandas as pd
import numpy as np

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [4]:
raw0 = pd.read_csv("data/2017-09-19_DB-Export_Freisetzungen.csv", encoding="iso-8859-1", error_bad_lines=False, engine="c", sep=";", low_memory=False)

In [5]:
#raw1 = raw0.loc[raw0.sector == 'Energy sector']
#raw2 = raw0.loc[np.logical_or(raw0.nace_sector == "Steam and air conditioning supply", raw0.nace_sector == "Production of electricity"), :]
raw2 = raw0.loc[:]

In [6]:
raw3 = raw2.sort_values("jahresfracht_freisetzung", ascending=True)

In [7]:
plants = raw3.rename(columns={"jahr": "year", "betriebsname_2": "plantname", "betriebsname": "company", "kennnummer": "plantid", "ort": "place", "strasse": "street", "bundesland": "federalstate", "jahresfracht_freisetzung": "amount"})

In [8]:
# plants

In [9]:
drop_list = ['flusseinzugsgebiet', 'schutzgrund_fracht', 'confidential_reason_release', 'schutzgrund_betrieb', 'confidential_reason_facility', 'versehentliche_freisetzung', 'bestimmungsmethode', 'determination_method', 'einheit', 'koord_rechtsw', 'koord_hochw', 'umweltkompartiment', 'geo_etrs89_utm32', 'haupttaetigkeit', 'taetigkeit', 'nace_wirtschaftszweig', 'schadstoff', 'stoffgruppe', 'branche', 'schadstoff_schwellenwert']
plants_2 = plants.drop(drop_list, axis=1, inplace=False)

In [10]:
#drop_list = ['company', 'plantname', 'hausnr']
#plants_2.drop(drop_list, axis=1, inplace=True)

In [11]:
#raw3.dtypes

In [12]:
#plants_3 = plants_2.loc[np.logical_and(plants_2.year == 2015, plants_2.plantid == "06-05-100-0248923"), :]
#plants_3 = plants_2.loc[np.logical_and(plants_2.year == 2015, plants_2.plantid == "06-05-100-0167182"), :]
# plants_4 = plants_3.loc[plants_3.company == "Stadtwerke Duisburg AG"]
# plants_3 = plants_2.loc[np.logical_and(plants_2.year == 2015, plants_2.pollutant == "Carbon dioxide (CO2)"), :]
plants_3 = plants_2.loc[:]
plants_4 = plants_3.loc[:]

In [13]:
# plants_4.sort_values("pollutant", ascending=False)

In [14]:
#plants_4.duplicated(subset=["year", "plantid", "pollutant"])

In [15]:
df = plants_4.loc[:]
btest = pd.concat(g for _, g in df.groupby(["year", "plantid", "pollutant"]) if len(g) > 1)

In [16]:
# btest.groupby("pollutant").count()

In [17]:
# plants_4.groupby(["year", "plantid", "pollutant"]).count()

In [18]:
plants_4.dtypes

year                  int64
plantid              object
company              object
plantname            object
plz                  object
place                object
street               object
hausnr               object
federalstate         object
taet_nr              object
activity             object
sector               object
nace_id               int64
nace_sector          object
substances_group     object
pollutant            object
releases_to          object
amount              float64
unit                 object
dtype: object

In [19]:
atest = plants_3.groupby(["pollutant"]).count()

In [20]:
# atest

In [21]:
def fix_company(company):
    cmp = company
    try:
        company_prefix, _ = company.split(" " , 1)
    except ValueError:
        return company
    # company_dict = {}
    company_dict = {"RWE": "RWE AG", "Vattenfall": "Vattenfall GmbH", "Uniper": "Uniper SE", "EnBW": "EnBW AG", "Steag": "Steag GmbH", "Nordzucker": "Nordzucker AG"}
    try:
        cmp = company_dict[company_prefix]
    except KeyError:
        return company
    return cmp

In [22]:
def fix_pollutant(pollutant):
    plt = pollutant
    if plt == "Trichlorobenzenes (TCBs) (all isomers)":
        return "TCBs"
    elif plt == "Phenols (as total C)":
        return "Phenols"
    elif plt == "Hydrochlorofluorocarbons(HCFCs)":
        return "HCFCs"
    elif plt == "Total organic carbon (TOC) (as total C or COD/3)":
        return "TCO"
    try:
        pollutant_list = pollutant.split(" ")
    except:
        return plt
    apollutant = pollutant_list.pop()
    if apollutant[:-2:-1] != ")":
        return apollutant
    elif apollutant[0] == '(':
        apollutant = apollutant[1:-1]
        apollutant = apollutant.split("/")[0]
        return apollutant
    else:
        return apollutant[0:-1]
    return plt

In [23]:
amount_dict = {"CO2": 9, "CO": 3, "SOx": 3, "NOx": 6, "PM10": 3, "HCl": 3, "HF": 3, "Benzene": 3}

In [24]:
unit_2_dict = {9: "Mio. t", 6: "Tsd. t", 3: "t"}

In [25]:
def convert_amount(amount, pollutant):
    try:
        pot = amount_dict[pollutant]
        unit_2 = unit_2_dict[pot]
        amount /= (10**pot)
    except KeyError:
        return amount, 1, "kg"
    return amount, pot, unit_2

In [26]:
convert_amount(3.210000e+10, "CO2")

(32.1, 9, 'Mio. t')

In [27]:
newdf = pd.DataFrame()
for index, row in plants_4.iterrows():
    row["plantid"] = row["plantid"].replace("/", "_")
    cmp = fix_company(row['company'])
    plt = fix_pollutant(row['pollutant'])
    #print(plt)
    if cmp == "RWE Power AG":
        print(cmp)
    amount, pot, unit_2 = convert_amount(row["amount"], plt)
    row['amount'] = amount
    row['potency'] = pot
    row['unit_2'] = unit_2
    row['company'] = cmp
    row['pollutant'] = plt
    newdf = newdf.append(row)

In [28]:
newdf= newdf[["plantid", "company", "plantname", "plz", "place", "street", "hausnr", "federalstate", "taet_nr", "activity", "sector", "nace_id", "nace_sector", "substances_group", "releases_to", "pollutant", "amount", 'potency', "unit", "unit_2", "year"]]

In [29]:
#newdf

In [30]:
# newdf.groupby(["pollutant"]).count()

In [31]:
drop_list = ['plz', 'company','plantname', 'federalstate',  'place', 'street', 'hausnr', 'taet_nr', 'nace_id', 'substances_group', 'unit', 'activity', 'sector', 'nace_sector' ]
pollutions_final = newdf.drop(drop_list, axis=1, inplace=False)

In [32]:
#print(plants_final.shape)
#print(plants_final.dtypes)

In [33]:
pollutions_final

,plantid,releases_to,pollutant,amount,potency,unit_2,year
13233,12-10651670000,Air,Teq,0.000101,1.0,kg,2009.0
39378,06-05-900-0877505,Air,Teq,0.000101,1.0,kg,2015.0
28507,03-09-09281734280,Water,Teq,0.000103,1.0,kg,2013.0
12905,06-11-01-1750001,Air,Teq,0.000105,1.0,kg,2009.0
4007,12-60510400000,Air,Teq,0.000107,1.0,kg,2007.0
30173,06-05-900-0045338,Air,Teq,0.000108,1.0,kg,2013.0
29016,06-05-100-0006538,Air,Teq,0.000110,1.0,kg,2013.0
36882,14-70-63809683116,Air,Teq,0.000110,1.0,kg,2014.0
11038,06-05-300-0923949,Air,Teq,0.000115,1.0,kg,2009.0
29290,06-05-100-0431554,Air,Teq,0.000116,1.0,kg,2013.0


In [34]:
#pl = newdf.dropna(subset=["plantid"])

In [35]:
pollutions_final['year'] = pollutions_final['year'].astype(int)
pollutions_final['potency'] = pollutions_final['potency'].astype(int)

In [36]:
pollutions_final.dtypes

plantid         object
releases_to     object
pollutant       object
amount         float64
potency          int64
unit_2          object
year             int64
dtype: object

In [37]:
pl0 = newdf.sort_values("plantname", ascending=False)

In [38]:
pl1 = pl0.drop_duplicates(subset="plantname")
pl2 = pl1.drop_duplicates(subset="plantid")
#plants_a = plants.drop_duplicates(subset="KraftwerkID")

In [39]:
pl2.dtypes

plantid              object
company              object
plantname            object
plz                  object
place                object
street               object
hausnr               object
federalstate         object
taet_nr              object
activity             object
sector               object
nace_id             float64
nace_sector          object
substances_group     object
releases_to          object
pollutant            object
amount              float64
potency             float64
unit                 object
unit_2               object
year                float64
dtype: object

In [40]:
#pl1.loc[pl1.plantname=="Kraftwerk Jänschwalde"]

In [41]:
#plt = pl2.loc[pl2.company == "RWE AG"]

In [42]:
#pl2

In [43]:
#pl = newdf.dropna(subset=["plantname"])

In [44]:
#pl.shape

In [45]:
#plants0 = newdf.groupby(["plantid"]).count()

In [46]:
#pl3 = plt.sort_values(["potency", "amount"], ascending=[False, False])

In [47]:
pl2

,plantid,company,plantname,plz,place,street,hausnr,federalstate,taet_nr,activity,...,nace_id,nace_sector,substances_group,releases_to,pollutant,amount,potency,unit,unit_2,year
2743,06-08-9839054,Zweckverband Abfallbehandlung Kahlenberg,z.H. Glen Tobiason,77975,Ringsheim,Bergwerkstrasse,1,Baden-Württemberg,5.c,Installations for the disposal of non-hazardou...,...,3821.0,Treatment and disposal of non-hazardous waste,Greenhouse gases,Air,CH4,5.940000e+05,1.0,kg/year,kg,2007.0
10902,06-05-300-0128149,Ford-Werke GmbH,"z. Hd. Herrn Stock, NL/A1",50735,Köln,Henry-Ford-Str.,1,Nordrhein-Westfalen,9.c,"Surface treatment of substances, objects or pr...",...,2910.0,Manufacture of motor vehicles,Other gases,Air,NMVOC,8.770000e+05,1.0,kg/year,kg,2009.0
6353,06-05-300-9023974,Wacker Chemie AG,vormals A P P bzw. Wacker Polymers,50769,Köln,Emdener Str.,117,Nordrhein-Westfalen,4.a,Chemical installations for the production on a...,...,2016.0,Manufacture of plastics in primary forms,Greenhouse gases,Air,HFCs,1.100000e+03,1.0,kg/year,kg,2008.0
5298,03-85-45300973450,"von Hammel GbR, Werner und Petra","von Hammel, Werner",49688,Lastrup,Sigge,1,Niedersachsen,7.a.i,Installations for the intensive rearing> 40 00...,...,147.0,Raising of poultry,Other gases,Air,NH3,1.340000e+04,1.0,kg/year,kg,2008.0
36654,13-30-396,v. Wendorff KG DaWa,v. Wendorff KG DaWa,17179,Walkendorf,Dorfstraße,59a,Mecklenburg-Vorpommern,7.a.ii,Installations for intensive rearing>2 000 plac...,...,146.0,Raising of swine/pigs,Other gases,Air,NH3,1.530000e+04,1.0,kg/year,kg,2014.0
25548,06-05-900-0205103,Gemeinschaftskraftwerk Bergkamen A OHG der STE...,und der RWE Power AG,59192,Bergkamen,Westenhellweg,110,Nordrhein-Westfalen,1.c,Thermal power stations and other combustion in...,...,3511.0,Production of electricity,Other gases,Air,SOx,2.590000e+03,3.0,kg/year,t,2012.0
19119,03-04-04040212960,Brüggen Oberflächen -,und Systemlieferant GmbH,49770,Herzlake,Boschstr.,4,Niedersachsen,2.f,Surface treatment of metals and plastic materi...,...,2920.0,Manufacture of bodies (coachwork) for motor ve...,Other gases,Air,NMVOC,1.310000e+05,1.0,kg/year,kg,2011.0
28268,03-01-01010251250,WAS Wolfsburger Abfallwirtschaft,u. Straßenreinigung,38442,Wolfsburg,Weyhäuser Weg,3,Niedersachsen,5.d,Landfills > Receiving 10 tonnes per day or wit...,...,3821.0,Treatment and disposal of non-hazardous waste,Greenhouse gases,Air,CH4,1.730000e+05,1.0,kg/year,kg,2013.0
30317,06-05-900-9000377,INNOVATHERM -Ges.zur innovativen Nu,tzung v.Brennstoffen mbH,44536,Lünen,Frydagstraße,47,Nordrhein-Westfalen,5.a,Installations for the recovery or disposal of ...,...,3822.0,Treatment and disposal of hazardous waste,Heavy metals,Water,Cu,1.380000e+02,1.0,kg/year,kg,2013.0
30781,06-08-9472002,Deponie DK0; DKII Schwaigern-Stetten,südlich von Stetten,74193,Schwaigern,Gurgelklinge,-,Baden-Württemberg,5.d,Landfills > Receiving 10 tonnes per day or wit...,...,3821.0,Treatment and disposal of non-hazardous waste,Greenhouse gases,Air,CH4,1.150000e+05,1.0,kg/year,kg,2013.0


In [48]:
drop_list = ['taet_nr', 'nace_id', 'substances_group', 'unit', 'activity', 'sector', 'nace_sector', 'pollutant', 'amount', 'potency', 'unit', 'unit_2', 'year']
plants0 = pl2.drop(drop_list, axis=1, inplace=False)

In [49]:
# plants0.sort_values(["potency", "amount"], ascending=[False, False])

In [50]:
plants0

,plantid,company,plantname,plz,place,street,hausnr,federalstate,releases_to
2743,06-08-9839054,Zweckverband Abfallbehandlung Kahlenberg,z.H. Glen Tobiason,77975,Ringsheim,Bergwerkstrasse,1,Baden-Württemberg,Air
10902,06-05-300-0128149,Ford-Werke GmbH,"z. Hd. Herrn Stock, NL/A1",50735,Köln,Henry-Ford-Str.,1,Nordrhein-Westfalen,Air
6353,06-05-300-9023974,Wacker Chemie AG,vormals A P P bzw. Wacker Polymers,50769,Köln,Emdener Str.,117,Nordrhein-Westfalen,Air
5298,03-85-45300973450,"von Hammel GbR, Werner und Petra","von Hammel, Werner",49688,Lastrup,Sigge,1,Niedersachsen,Air
36654,13-30-396,v. Wendorff KG DaWa,v. Wendorff KG DaWa,17179,Walkendorf,Dorfstraße,59a,Mecklenburg-Vorpommern,Air
25548,06-05-900-0205103,Gemeinschaftskraftwerk Bergkamen A OHG der STE...,und der RWE Power AG,59192,Bergkamen,Westenhellweg,110,Nordrhein-Westfalen,Air
19119,03-04-04040212960,Brüggen Oberflächen -,und Systemlieferant GmbH,49770,Herzlake,Boschstr.,4,Niedersachsen,Air
28268,03-01-01010251250,WAS Wolfsburger Abfallwirtschaft,u. Straßenreinigung,38442,Wolfsburg,Weyhäuser Weg,3,Niedersachsen,Air
30317,06-05-900-9000377,INNOVATHERM -Ges.zur innovativen Nu,tzung v.Brennstoffen mbH,44536,Lünen,Frydagstraße,47,Nordrhein-Westfalen,Water
30781,06-08-9472002,Deponie DK0; DKII Schwaigern-Stetten,südlich von Stetten,74193,Schwaigern,Gurgelklinge,-,Baden-Württemberg,Air


In [51]:
plants0.dtypes

plantid         object
company         object
plantname       object
plz             object
place           object
street          object
hausnr          object
federalstate    object
releases_to     object
dtype: object

In [52]:
pollutions_final

,plantid,releases_to,pollutant,amount,potency,unit_2,year
13233,12-10651670000,Air,Teq,0.000101,1,kg,2009
39378,06-05-900-0877505,Air,Teq,0.000101,1,kg,2015
28507,03-09-09281734280,Water,Teq,0.000103,1,kg,2013
12905,06-11-01-1750001,Air,Teq,0.000105,1,kg,2009
4007,12-60510400000,Air,Teq,0.000107,1,kg,2007
30173,06-05-900-0045338,Air,Teq,0.000108,1,kg,2013
29016,06-05-100-0006538,Air,Teq,0.000110,1,kg,2013
36882,14-70-63809683116,Air,Teq,0.000110,1,kg,2014
11038,06-05-300-0923949,Air,Teq,0.000115,1,kg,2009
29290,06-05-100-0431554,Air,Teq,0.000116,1,kg,2013


In [53]:
# plants0.to_csv("plants.csv", index=False, header=False)
pollutions_final.to_csv("pollutions_nh.csv", index=False, header=False)
# blocks.to_csv("blocks_nh.csv", index=False, header=False)